## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-06-05-51-39 +0000,bbc,Trump sends troops from California to Portland...,https://www.bbc.com/news/articles/c740elm70z7o...
1,2025-10-06-05-38-40 +0000,nyt,Judge Again Blocks Guard Deployment as Trump E...,https://www.nytimes.com/2025/10/05/us/trump-na...
2,2025-10-06-04-55-00 +0000,wsj,Judge Blocks Trump from Deploying to Oregon An...,https://www.wsj.com/politics/policy/california...
3,2025-10-06-04-40-45 +0000,nypost,Trump to send 400 Texas National Guard troops ...,https://nypost.com/2025/10/06/us-news/trump-se...
4,2025-10-06-04-37-24 +0000,nyt,"Drones, Democracy and the War in Ukraine",https://www.nytimes.com/2025/10/05/world/ukain...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,37
33,new,12
119,gaza,11
30,war,10
10,guard,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,138
105,2025-10-05-18-37-30 +0000,latimes,Judge blocks Trump from deploying California N...,https://www.latimes.com/world-nation/story/202...,90
115,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...,85
2,2025-10-06-04-55-00 +0000,wsj,Judge Blocks Trump from Deploying to Oregon An...,https://www.wsj.com/politics/policy/california...,78
15,2025-10-06-03-00-00 +0000,wsj,The Supreme Court officially returns for a new...,https://www.wsj.com/politics/policy/supreme-co...,78


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,138,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
15,41,2025-10-06-03-00-00 +0000,wsj,The Supreme Court officially returns for a new...,https://www.wsj.com/politics/policy/supreme-co...
114,35,2025-10-05-17-31-44 +0000,nypost,Gaza flotilla activists allege Israeli soldier...,https://nypost.com/2025/10/05/world-news/gaza-...
73,31,2025-10-05-20-59-08 +0000,nypost,Jersey City Mayor Steve Fulop being eyed for b...,https://nypost.com/2025/10/05/us-news/jersey-c...
52,29,2025-10-05-22-51-18 +0000,nypost,Chicago police ordered to ignore border patrol...,https://nypost.com/2025/10/05/us-news/chicago-...
134,27,2025-10-05-14-45-55 +0000,nypost,Virginia Beach Dems back Virginia AG nominee J...,https://nypost.com/2025/10/05/us-news/dems-bac...
186,27,2025-10-05-07-00-00 +0000,wapo,AI lets Russian war widows see their soldier h...,https://www.washingtonpost.com/world/2025/10/0...
167,27,2025-10-05-09-30-00 +0000,wsj,"As Laura Loomer, an influential loyalist to Pr...",https://www.wsj.com/politics/policy/laura-loom...
70,25,2025-10-05-21-24-00 +0000,wsj,The federal government shutdown is dragging to...,https://www.wsj.com/politics/policy/government...
65,24,2025-10-05-21-33-27 +0000,nypost,NJ high school teacher charged with sexually a...,https://nypost.com/2025/10/05/us-news/nj-high-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
